In [1]:
# STEP 1: Import libraries
import pandas as pd
import sqlite3


In [2]:
# STEP 2: Create SQLite database
conn = sqlite3.connect("food_delivery.db")


In [3]:
# STEP 3: Load CSV data (orders)
orders = pd.read_csv("orders.csv")
orders.to_sql("orders", conn, if_exists="replace", index=False)


10000

In [4]:
# STEP 4: Load JSON data (users)
users = pd.read_json("users.json")
users.to_sql("users", conn, if_exists="replace", index=False)


3000

In [5]:
# STEP 5: Load SQL data (restaurants)
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)


In [6]:
# STEP 6: Perform LEFT JOIN (retain all orders)
query = """
SELECT *
FROM orders
LEFT JOIN users
ON orders.user_id = users.user_id
LEFT JOIN restaurants
ON orders.restaurant_id = restaurants.restaurant_id
"""


In [7]:
final_df = pd.read_sql(query, conn)


In [8]:
# STEP 8: Save final output file
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [9]:
# STEP 9: Preview final dataset
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,user_id,name,city,membership,restaurant_id,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,2508,User_2508,Hyderabad,Regular,450,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,2693,User_2693,Pune,Regular,309,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,2084,User_2084,Chennai,Gold,107,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,319,User_319,Bangalore,Gold,224,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,1064,User_1064,Pune,Regular,293,Restaurant_293,Italian,3.0


In [11]:
query = """
SELECT *
FROM orders
LEFT JOIN users
ON orders.user_id = users.user_id
LEFT JOIN restaurants
ON orders.restaurant_id = restaurants.restaurant_id
"""


In [12]:
final_df = pd.read_sql(query, conn)
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,user_id,name,city,membership,restaurant_id,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,2508,User_2508,Hyderabad,Regular,450,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,2693,User_2693,Pune,Regular,309,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,2084,User_2084,Chennai,Gold,107,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,319,User_319,Bangalore,Gold,224,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,1064,User_1064,Pune,Regular,293,Restaurant_293,Italian,3.0


In [13]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [14]:
final_df.groupby('cuisine')['total_amount'] \
        .mean() \
        .sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [17]:
final_df.groupby('city')['total_amount'].sum()


,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Hyderabad,1889366.58
Pune,1924797.93


In [18]:
final_df.groupby('cuisine')['total_amount'].mean()


,total_amount
cuisine,
Chinese,798.389020
Indian,798.466011
Italian,799.448578
Mexican,808.021344


In [19]:
final_df.groupby('membership')['order_id'].count()


,order_id
membership,
Gold,4987
Regular,5013


In [67]:
# Drop duplicate columns if they exist (caused by SQL JOIN with SELECT * and subsequent loading)
if 'user_id.1' in final_df.columns:
    final_df = final_df.drop(columns=['user_id.1'])
if 'restaurant_id.1' in final_df.columns:
    final_df = final_df.drop(columns=['restaurant_id.1'])
if 'restaurant_name.1' in final_df.columns:
    final_df = final_df.drop(columns=['restaurant_name.1'])

user_total = final_df.groupby('user_id')['total_amount'].sum()

count_users = (user_total > 1000).sum()
count_users

np.int64(2544)

In [22]:
final_df.groupby(pd.cut(final_df['rating'], [3.0,3.5,4.0,4.5,5.0], labels=['3.0–3.5','3.6–4.0','4.1–4.5','4.6–5.0']))['total_amount'].sum().idxmax()


/tmp/ipython-input-2117644966.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby(pd.cut(final_df['rating'], [3.0,3.5,4.0,4.5,5.0], labels=['3.0–3.5','3.6–4.0','4.1–4.5','4.6–5.0']))['total_amount'].sum().idxmax()


'4.6–5.0'

In [23]:
final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()


'Chennai'

In [24]:
final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()


'Chennai'

In [34]:
# Ensure date is datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

# Group by quarter and sum revenue, then get quarter with highest revenue
(final_df.groupby(final_df['order_date'].dt.quarter)['total_amount'].sum()
         .sort_values(ascending=False)
         .head(1))

,total_amount
order_date,
3,2037385.1


In [36]:
(final_df.groupby(['membership','cuisine'])['total_amount'].sum()
         .sort_values(ascending=False)
         .head(1))


,,total_amount
membership,cuisine,
Regular,Mexican,1072943.3


In [37]:
' + '.join((final_df.groupby(['membership','cuisine'])['total_amount'].sum()
                       .sort_values(ascending=False)
                       .head(1)
                       .index[0]))


'Regular + Mexican'

In [38]:
import pandas as pd

# Reload the CSV into final_df
final_df = pd.read_csv('final_food_delivery_dataset.csv')

# Optional: check the first few rows
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,user_id.1,name,city,membership,restaurant_id.1,restaurant_name.1,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,2508,User_2508,Hyderabad,Regular,450,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,2693,User_2693,Pune,Regular,309,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,2084,User_2084,Chennai,Gold,107,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,319,User_319,Bangalore,Gold,224,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,1064,User_1064,Pune,Regular,293,Restaurant_293,Italian,3.0


In [40]:
# Save the current state of final_df as a CSV
final_df.to_csv('final_df_backup.csv', index=False)


In [41]:
' + '.join((final_df.groupby(['membership','cuisine'])['total_amount'].sum()
                        .sort_values(ascending=False)
                        .head(1)
                        .index[0]))


'Regular + Mexican'

In [42]:
import pandas as pd

# 1️⃣ Ensure correct types
final_df['membership'] = final_df['membership'].astype(str)
final_df['cuisine'] = final_df['cuisine'].astype(str)
final_df['total_amount'] = final_df['total_amount'].astype(float)

# 2️⃣ Define only the combinations we care about
options = [('Gold','Indian'), ('Gold','Italian'), ('Regular','Indian'), ('Regular','Chinese')]

# 3️⃣ Filter the dataframe for these combinations
filtered_df = final_df[final_df[['membership','cuisine']].apply(tuple, axis=1).isin(options)]

# 4️⃣ Group by membership + cuisine, sum revenue, and get the highest
highest_combo = filtered_df.groupby(['membership','cuisine'])['total_amount'].sum().sort_values(ascending=False).head(1)

# 5️⃣ Show just the combination as string
' + '.join(highest_combo.index[0])


'Gold + Italian'

In [43]:
total_orders = len(final_df)
gold_orders = len(final_df[final_df['membership'] == 'Gold'])

percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [44]:
final_df.groupby('cuisine').agg(
    restaurant_count=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).sort_values('restaurant_count')


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [45]:
restaurant_stats = final_df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

restaurant_stats[restaurant_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False)


,avg_order_value,total_orders
restaurant_name,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [46]:
final_df.groupby(['membership', 'cuisine'])['total_amount'] \
        .sum() \
        .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [48]:
# 1️⃣ Ensure correct types
final_df['restaurant_id'] = final_df['restaurant_id'].astype(str)
final_df['total_amount'] = final_df['total_amount'].astype(float)

# 2️⃣ Group by restaurant, compute total orders and average order value
restaurant_stats = final_df.groupby(['restaurant_id','restaurant_name']).agg(
    total_orders=('order_id','count'),
    avg_order_value=('total_amount','mean')
)

# 3️⃣ Filter restaurants with less than 20 orders
filtered_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]

# 4️⃣ Get the restaurant with highest average order value
filtered_restaurants.sort_values(by='avg_order_value', ascending=False).head(1)


,,total_orders,avg_order_value
restaurant_id,restaurant_name,,
294,Hotel Dhaba Multicuisine,13,1040.222308


In [49]:
filtered_restaurants.sort_values(by='avg_order_value', ascending=False).head(1).index[0][1]


'Hotel Dhaba Multicuisine'

In [50]:
# List of options
options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian',
           'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']

# Filter only these restaurants
filtered_df = final_df[final_df['restaurant_name'].isin(options)]

# Group by restaurant, compute total orders and average order value
restaurant_stats = filtered_df.groupby('restaurant_name').agg(
    total_orders=('order_id','count'),
    avg_order_value=('total_amount','mean')
)

# Filter restaurants with less than 20 orders
restaurant_stats = restaurant_stats[restaurant_stats['total_orders'] < 20]

# Get the one with highest average order value
restaurant_stats.sort_values(by='avg_order_value', ascending=False).head(1)


,total_orders,avg_order_value
restaurant_name,,
Ruchi Foods Chinese,19,686.603158


In [51]:
restaurant_stats.sort_values(by='avg_order_value', ascending=False).head(1).index[0]


'Ruchi Foods Chinese'

In [54]:
# Ensure membership column is string
final_df['membership'] = final_df['membership'].astype(str)

# Count total orders by Gold members
total_gold_orders = final_df[final_df['membership'] == 'Gold']['order_id'].count()

total_gold_orders


np.int64(4987)

In [53]:
total_gold_orders = int(total_gold_orders)  # optional


In [55]:

# Ensure correct type
final_df['total_amount'] = final_df['total_amount'].astype(float)
final_df['city'] = final_df['city'].astype(str)

# Sum total_amount for Hyderabad orders and round
total_revenue_hyderabad = round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum())

total_revenue_hyderabad


1889367

In [56]:
# Ensure user_id is string
final_df['user_id'] = final_df['user_id'].astype(str)

# Count distinct users
distinct_users = final_df['user_id'].nunique()

distinct_users


2883

In [57]:
# Ensure total_amount column is float
final_df['total_amount'] = final_df['total_amount'].astype(float)
final_df['membership'] = final_df['membership'].astype(str)

# Compute average order value for Gold members
avg_order_value_gold = round(final_df[final_df['membership'] == 'Gold']['total_amount'].mean(), 2)

avg_order_value_gold


np.float64(797.15)

In [58]:
# Ensure rating is float
final_df['rating'] = final_df['rating'].astype(float)

# Count orders where restaurant rating >= 4.5
orders_high_rating = final_df[final_df['rating'] >= 4.5]['order_id'].count()

orders_high_rating


np.int64(3374)

In [59]:
# Ensure correct types
final_df['membership'] = final_df['membership'].astype(str)
final_df['city'] = final_df['city'].astype(str)

# 1️⃣ Filter only Gold members
gold_df = final_df[final_df['membership'] == 'Gold']

# 2️⃣ Find the city with highest total revenue among Gold members
top_city = gold_df.groupby('city')['total_amount'].sum().idxmax()

# 3️⃣ Count orders in that city
orders_in_top_city = gold_df[gold_df['city'] == top_city]['order_id'].count()

orders_in_top_city


np.int64(1337)

In [60]:
import pandas as pd

# Load the files
orders = pd.read_csv('orders.csv')
users = pd.read_json('users.json')

# Show common columns
common_columns = orders.columns.intersection(users.columns)
common_columns


Index(['user_id'], dtype='object')

In [63]:
# Total number of rows
total_rows = final_df.shape[0]

total_rows


10000

In [64]:
final_df = orders.merge(users, on='user_id', how='left')


In [66]:
import pandas as pd

orders = pd.read_csv('orders.csv')
users = pd.read_json('users.json')

# Left join
final_df = orders.merge(users, on='user_id', how='left')

# Show rows where user info is missing
final_df[final_df['name'].isna()].head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
